In [1]:
import os
from collections import defaultdict

import numpy as np
import pandas as pd
import rioxarray as rio

# Average exceedances

See e-mail from Kari received 22.06.2023 at 12.45. This notebook calculates the mean of all cells where the exceedance is greater than zero.

In [2]:
# User options
cell_size = 120

# Series to use
series_dict = {
    "7882": (1978, 1982),
    "8387": (1983, 1987),
    "8892": (1988, 1992),
    "9296": (1992, 1996),
    "9701": (1997, 2001),
    "0206": (2002, 2006),
    "0711": (2007, 2011),
    "1216_old": (2012, 2016),
    "1721_old": (2017, 2021),
    "1721_blrgrid": (2017, 2021),
    "1721_metgrid": (2017, 2021),
}

# Exceedances to consider
model_list = ["soil", "veg", "sswc", "fab"]

In [3]:
def build_path(base_path, series, model, cell_size):
    """ """
    if model == "soil":
        fname = f"soil_ex_mgpm2pyr_{series}_{cell_size}m.tif"
    elif model == "veg":
        fname = f"veg_ex_mgpm2pyr_{series}_{cell_size}m.tif"
    elif model == "sswc":
        fname = f"sswc_ex_meqpm2pyr_{series}_{cell_size}m.tif"
    elif model == "fab":
        fname = f"fab_exns_meqpm2pyr_{series}_{cell_size}m.tif"
    else:
        raise ValueError("'model' is not valid.")

    return os.path.join(base_path, series, fname)

In [4]:
base_path = r"/home/jovyan/shared/critical_loads/raster/exceedance"
data = defaultdict(list)
for series, (st_yr, end_yr) in series_dict.items():
    for model in model_list:
        # Read data
        fname = build_path(base_path, series, model, cell_size)
        ds = rio.open_rasterio(fname, mask_and_scale=True)

        # Get exceedances
        ex_area = (ds > 0).sum().item() * cell_size * cell_size / 1e6
        tot_area = ds.notnull().sum().item() * cell_size * cell_size / 1e6
        avg_ex = ds.where(ds > 0).mean().item()

        # FAB and SSWC are already in meq/m2/yr; soil and veg are in mg/m2/yr
        if model == "fab":
            medium = "water_fab"
        elif model == "sswc":
            medium = "water_sswc"
        elif model == "veg":
            # Only based on N. Convert to meq/m2/yr
            medium = "vegetation"
            avg_ex = avg_ex / 14.01
        elif model == "soil":
            # Only based on S. Convert to meq/m2/yr
            medium = "soil"
            avg_ex = avg_ex * 2 / 32.06
        else:
            raise ValueError("'model' is not valid.")

        data["short_name"].append(series)
        data["medium"].append(medium)
        # data["tot_area_km2"].append(round(tot_area))
        # data["ex_area_km2"].append(round(ex_area))
        data["avg_ex_meqpm2pyr"].append(round(avg_ex, 1))

df = pd.DataFrame(data)

# Save
out_csv = r"../../summaries/average_exceedance_june_2023.csv"
df.to_csv(out_csv, index=False)

df

,short_name,medium,avg_ex_meqpm2pyr
0,7882,soil,10.1
1,7882,vegetation,26.9
2,7882,water_sswc,33.8
3,7882,water_fab,53.8
4,8387,soil,10.4
5,8387,vegetation,38.2
6,8387,water_sswc,33.0
7,8387,water_fab,63.9
8,8892,soil,11.4
9,8892,vegetation,35.9
